# Loading libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectFromModel
import xgboost as xg


import missingno as msno 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import KMeans, DBSCAN
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet
from  sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from numpy import log
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import multiprocessing
from sklearn.model_selection import KFold
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_validate






#from sklearn.utils import resample
%matplotlib inline
sns.set_style("darkgrid")
from sklearn.metrics import mean_squared_error

# Importing the dataset

In [4]:
new_df_dies = pd.read_csv('new_df_dies.csv').set_index('Dates')
df_prova = pd.read_csv('df_prova.csv')
df = pd.read_csv('df.csv')

df['month_year'] = pd.to_datetime(df.Dates).dt.to_period('M')
df['Dates'] = pd.to_datetime(df['Date']).dt.date
df['Time'] = pd.to_datetime(df['Date']).dt.time

selected_types = ['CRIMINAL SEXUAL ASSAULT', 'SEX OFFENSE']
descriptions = ['PUBLIC INDECENCY', 'SEXUAL EXPLOITATION OF A CHILD', 
       'INDECENT SOLICITATION/CHILD', 'CRIMINAL TRANSMISSION OF HIV',
       'INDECENT SOLICITATION OF A CHILD', 'FORNICATION', 'BIGAMY',
        'MARRYING A BIGAMIST']
df.Dates = pd.to_datetime(df.Dates)
df_prova = df[(df['Primary Type'] == 'CRIMINAL SEXUAL ASSAULT') | (df['Primary Type'] == 'SEX OFFENSE')]
for description in descriptions:
    df_prova = df_prova[df_prova['Description'] != description]

In [5]:
new_df_dies.index = pd.to_datetime(new_df_dies.index)

# Setting main dates

In [6]:
date ={}
date['date_start_train'] = '2001-01-01'
date['date_end_train_mesos'] = '2022-12-17'

date['date_start_test_mesos'] = '2023-01-23'
date['date_end_test_mesos'] = '2023-02-06'

In [7]:
# !pip install gluonts
# !pip install mxnet

In [8]:
%matplotlib inline
import plotly.express as px
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as sm
from statsmodels.tsa.stattools import kpss ,  adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import mxnet as mx


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gluonts
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer import Trainer
import matplotlib.pyplot  as plt
import pandas as pd
import matplotlib as mpl

from mxnet import gpu

from gluonts.evaluation import Evaluator


In [9]:
new_df_dies.keys()

Index(['Agressions', 'New Years', 'agressions_lag_1', 'agressions_lag_2',
       'agressions_lag_3', 'agressions_lag_4', 'agressions_week_avg',
       'agressions_biweek_avg', 'agressions_1_month_avg',
       'agressions_2_month_avg', 'month', 'day', 'weekofyear', 'isweekend',
       'firstday', 'primavera', 'estiu', 'tardor', 'hivern', 'daysinmonth',
       'year_2002', 'year_2003', 'year_2004', 'year_2005', 'year_2006',
       'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011',
       'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016',
       'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021',
       'year_2022', 'year_2023', 'quarter_2', 'quarter_3', 'quarter_4',
       'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4',
       'dayofweek_5', 'dayofweek_6', 'trend', 'sin(1,freq=A-DEC)',
       'cos(1,freq=A-DEC)', 'sin(2,freq=A-DEC)', 'cos(2,freq=A-DEC)',
       'sin(3,freq=A-DEC)', 'cos(3,freq=A-DEC)', 'sin(4,freq=A-DEC)',
       'cos(

In [10]:
from sklearn.preprocessing import OrdinalEncoder

def add_header_keys(df, keys, header):
    map_ = {k: header + k for k in keys}
    return df.rename(columns=map_)


def split_dynamic_stat_data(df, keys=None, thresh=0.05):
    if keys is None:
        keys = [k for k in df.keys() if k not in ['Agressions', 'code', 'ds']]
    dynamic = []
    unique_codes = np.unique(df.index)
    limit = int(thresh * len(unique_codes))
    counts = {k: [] for k in keys}
    for k in keys:
        if len(np.unique(df[k])) > limit:
                dynamic.append(k)
                keys.remove(k)
    static = [k for k in keys]

    return dynamic, static



def split_real_categorical_data(df, thresh=0.1, keys=[], isdynamical=True, auto=False):
    """
    To decide if a variable is categorical or not, we can do the following.
    Take the length of all the values in a series and compare it with the unique values of the series.
    If the unique values is less than 10% of the length of the full series, it means that the variable is categorical.
    Else, the variable is dynamic.
    This nemotecnic rule should work pretty good.
    """

    if len(keys) == 0 and auto:
        keys = [k for k in df.keys() if k not in ['Agressions', 'code', 'ds']]
    elif len(keys) == 0:
        return [], []

    if isdynamical:
        """
        this will not work for statical data... but it is the most pure way to do it.
        """
        votes = {k: 0 for k in keys}
        for k in keys:
            if len(df[k]) > 1:
                if len(np.unique(df[k])) < thresh * len(df[k]) or len(np.unique(df[k])) == 1:
                    votes[k] -= 1
                else:
                    votes[k] += 1.5

        categorical = []
        real = []
        for k in votes.keys():
            if votes[k] >= 0:
                real.append(k)
            else:
                categorical.append(k)

    else:
        categorical = []
        for k in keys:
            if len(np.unique(df[k])) < thresh * len(df):
                categorical.append(k)
                keys.remove(k)
        real = keys

    print(f'Real keys before formatting {real}')
    print(f'Categorical keys before formatting {categorical}')
    print()
    for k in real:
        if isinstance(np.array(df[k])[0], str):
            categorical.append(k)
            real.remove(k)
    for k in categorical:
        if isinstance(np.array(df[k])[0], float):
            real.append(k)
            categorical.remove(k)
    print(f'Real keys after formatting {real}')
    print(f'Categorical keys after formatting {categorical}')
    return categorical, real



def split_features(df, thresh=0.1):
    dyn, stat = split_dynamic_stat_data(df)
    dyn_cat, dyn_re = split_real_categorical_data(df, thresh=thresh, keys=dyn)
    stat_cat, stat_re = split_real_categorical_data(df, thresh=thresh, keys=stat)
    return dyn_cat, dyn_re, stat_cat, stat_re

def generate_labelled_exogenous_df(df, one_hot=True, thresh=0.1,ignore_keys = [['Agressions']]):
    df = df.fillna(0)
    dyn_cat, dyn_re, stat_cat, stat_re = split_features(df, thresh=thresh)
    print('Dynamical categorical features:')
    for k in dyn_cat:
        print(k)
    print('\n\n')
    print('Dynamical real features:')
    for k in dyn_re:
        print(k)
    print('\n\n')
    print('Statical categorical features:')
    for k in stat_cat:
        print(k)
    print('\n\n')
    print('Statical real features:')
    for k in stat_re:
        print(k)
    print('\n\n')
    new_df = df.copy()
    new_df = add_header_keys(new_df, dyn_cat, 'dyn_cat_')
    new_df = add_header_keys(new_df, dyn_re, 'dyn_re_')
    new_df = add_header_keys(new_df, stat_cat, 'stat_cat_')
    new_df = add_header_keys(new_df, stat_re, 'stat_re_')

    ordinal_encoder = None

    if one_hot:
        cat_dyn_keys = [k for k in new_df.keys() if 'dyn_cat_' in k]
        for k in cat_dyn_keys:
            tmp = pd.get_dummies(new_df[k], prefix=k)
            for new_k in tmp.keys():
                new_k_re = new_k.replace('dyn_cat_', 'dyn_re_OneHot_')
                new_df[new_k_re] = tmp[new_k].copy()
        new_df = new_df.drop(cat_dyn_keys, axis=1)

        cat_sat_keys = [k for k in new_df.keys() if 'stat_cat_' in k]
        features = new_df[cat_sat_keys].to_numpy()
        ordinal_encoder = OrdinalEncoder().fit(features)
        encoded_features = ordinal_encoder.transform(features)
        for i, k in enumerate(cat_sat_keys):
            new_df[k] = encoded_features[:, i]

    return new_df, ordinal_encoder

In [11]:
dd, encod = generate_labelled_exogenous_df(new_df_dies, one_hot = False)

Real keys before formatting ['trend']
Categorical keys before formatting ['cos(1,freq=A-DEC)', 'cos(2,freq=A-DEC)', 'cos(3,freq=A-DEC)', 'cos(4,freq=A-DEC)', 'cos(5,freq=A-DEC)']

Real keys after formatting ['trend', 'cos(1,freq=A-DEC)', 'cos(3,freq=A-DEC)', 'cos(5,freq=A-DEC)']
Categorical keys after formatting ['cos(2,freq=A-DEC)', 'cos(4,freq=A-DEC)']
Real keys before formatting []
Categorical keys before formatting ['New Years', 'agressions_lag_1', 'agressions_lag_2', 'agressions_lag_3', 'agressions_lag_4', 'agressions_week_avg', 'agressions_biweek_avg', 'agressions_1_month_avg', 'agressions_2_month_avg', 'month', 'day', 'weekofyear', 'isweekend', 'firstday', 'primavera', 'estiu', 'tardor', 'hivern', 'daysinmonth', 'year_2002', 'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022'

In [12]:
dd

,Agressions,stat_cat_New Years,stat_re_agressions_lag_1,stat_cat_agressions_lag_2,stat_re_agressions_lag_3,stat_cat_agressions_lag_4,stat_re_agressions_week_avg,stat_cat_agressions_biweek_avg,stat_re_agressions_1_month_avg,stat_cat_agressions_2_month_avg,...,"stat_cat_sin(1,freq=W-SUN)","stat_re_cos(1,freq=W-SUN)","stat_cat_sin(2,freq=W-SUN)","stat_re_cos(2,freq=W-SUN)","stat_cat_cos(3,freq=W-SUN)","stat_re_sin(4,freq=W-SUN)",stat_cat_Poverty,stat_re_Foreign born,stat_cat_HS Grads,stat_re_BA Grads
Dates,,,,,,,,,,,,,,,,,,,,,
2001-01-01,104.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.196,0.217,0.718,0.255
2001-01-02,5.0,0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.781831,0.623490,0.974928,-0.222521,-0.900969,-0.433884,0.196,0.217,0.718,0.255
2001-01-03,11.0,0,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.974928,-0.222521,-0.433884,-0.900969,0.623490,0.781831,0.196,0.217,0.718,0.255
2001-01-04,3.0,0,11.0,5.0,104.0,0.0,0.0,0.0,0.0,0.0,...,0.433884,-0.900969,-0.781831,0.623490,-0.222521,-0.974928,0.196,0.217,0.718,0.255
2001-01-05,8.0,0,3.0,11.0,5.0,104.0,0.0,0.0,0.0,0.0,...,-0.433884,-0.900969,0.781831,0.623490,-0.222521,0.974928,0.196,0.217,0.718,0.255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-02,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.433884,-0.900969,-0.781831,0.623490,-0.222521,-0.974928,0.173,0.202,0.859,0.411
2023-02-03,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.433884,-0.900969,0.781831,0.623490,-0.222521,0.974928,0.173,0.202,0.859,0.411
2023-02-04,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.974928,-0.222521,0.433884,-0.900969,0.623490,-0.781831,0.173,0.202,0.859,0.411


In [13]:
cols_dyn_re = dd.columns[dd.columns.str.match(r'dyn_re')]
cols_stat_cat = dd.columns[dd.columns.str.match(r'stat_cat')]
cols_dyn_cat = dd.columns[dd.columns.str.match(r'dyn_cat')]
cols_stat_re = dd.columns[dd.columns.str.match(r'stat_re')]

In [14]:
len(cols_dyn_re),len(cols_stat_cat),len(cols_dyn_cat),len(cols_stat_re)

(4, 55, 2, 14)

In [15]:
cols_dyn_re = cols_dyn_re.tolist() + cols_dyn_cat.tolist()

In [16]:
cols_stat_cat.tolist()

['stat_cat_New Years',
 'stat_cat_agressions_lag_2',
 'stat_cat_agressions_lag_4',
 'stat_cat_agressions_biweek_avg',
 'stat_cat_agressions_2_month_avg',
 'stat_cat_month',
 'stat_cat_day',
 'stat_cat_weekofyear',
 'stat_cat_isweekend',
 'stat_cat_firstday',
 'stat_cat_primavera',
 'stat_cat_estiu',
 'stat_cat_tardor',
 'stat_cat_hivern',
 'stat_cat_daysinmonth',
 'stat_cat_year_2002',
 'stat_cat_year_2003',
 'stat_cat_year_2004',
 'stat_cat_year_2005',
 'stat_cat_year_2006',
 'stat_cat_year_2007',
 'stat_cat_year_2008',
 'stat_cat_year_2009',
 'stat_cat_year_2010',
 'stat_cat_year_2011',
 'stat_cat_year_2012',
 'stat_cat_year_2013',
 'stat_cat_year_2014',
 'stat_cat_year_2015',
 'stat_cat_year_2016',
 'stat_cat_year_2017',
 'stat_cat_year_2018',
 'stat_cat_year_2019',
 'stat_cat_year_2020',
 'stat_cat_year_2021',
 'stat_cat_year_2022',
 'stat_cat_year_2023',
 'stat_cat_quarter_2',
 'stat_cat_quarter_3',
 'stat_cat_quarter_4',
 'stat_cat_dayofweek_1',
 'stat_cat_dayofweek_2',
 'stat_ca

In [17]:
cardinality = dd[cols_stat_cat.tolist()].max()+1

In [18]:
cardinality = cardinality.astype(int).tolist()

In [19]:
dd.keys()

Index(['Agressions', 'stat_cat_New Years', 'stat_re_agressions_lag_1',
       'stat_cat_agressions_lag_2', 'stat_re_agressions_lag_3',
       'stat_cat_agressions_lag_4', 'stat_re_agressions_week_avg',
       'stat_cat_agressions_biweek_avg', 'stat_re_agressions_1_month_avg',
       'stat_cat_agressions_2_month_avg', 'stat_cat_month', 'stat_cat_day',
       'stat_cat_weekofyear', 'stat_cat_isweekend', 'stat_cat_firstday',
       'stat_cat_primavera', 'stat_cat_estiu', 'stat_cat_tardor',
       'stat_cat_hivern', 'stat_cat_daysinmonth', 'stat_cat_year_2002',
       'stat_cat_year_2003', 'stat_cat_year_2004', 'stat_cat_year_2005',
       'stat_cat_year_2006', 'stat_cat_year_2007', 'stat_cat_year_2008',
       'stat_cat_year_2009', 'stat_cat_year_2010', 'stat_cat_year_2011',
       'stat_cat_year_2012', 'stat_cat_year_2013', 'stat_cat_year_2014',
       'stat_cat_year_2015', 'stat_cat_year_2016', 'stat_cat_year_2017',
       'stat_cat_year_2018', 'stat_cat_year_2019', 'stat_cat_year_2020'

In [23]:
mx.random.seed(7)
np.random.seed(7)

In [24]:

class DeepAR() : 
    def __init__(self,df) : 

        """ 
        init (self,df) : the constructor of the class

        """
        self.df = df
    
    def prepare_dataset(self,start_index,target_index,target,freq):
        dataset = ListDataset([{"start": start_index, 
                                            "target": self.df[target][start_index:target_index]}], freq = freq)
        return dataset
    
    def prepare_dataset_grouped(self,start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list):
        dataset_list = []
        current_dataset = {"start": start_index, 
                                        "target": self.df[target][start_index:target_index]}

        dynre_list = []
        statcat_list = []
        statre_list = []

        for dyn_re in dyn_re_list:
            dynre_list.append(self.df[dyn_re][start_index:target_index])

        for stat_cat in stat_cat_list:
            statcat_list.append(self.df[stat_cat].iloc[0])
            
        for stat_re in stat_re_list:
            statre_list.append(self.df[stat_re].iloc[0])
#             statre_list.append(self.df[start_index:target_index])



        current_dataset[FieldName.FEAT_STATIC_CAT] = statcat_list
        current_dataset[FieldName.FEAT_DYNAMIC_REAL] = dynre_list
        current_dataset[FieldName.FEAT_STATIC_REAL] = statre_list

        dataset_list.append(current_dataset)



        dataset = ListDataset(dataset_list, freq = freq)

        return dataset  


    def prepare_train (self,start_index,target_index,target, freq, dyn_re_list, stat_cat_list, stat_re_list) : 
        self.training_data = self.prepare_dataset_grouped(start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list)

    def prepare_test (self,start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list) : 

        self.test_data = self.prepare_dataset_grouped(start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list)

    def estimator (self,freq,context_length,prediction_length,num_layers,num_cells,cell_type,epochs,cardinality, use_external = True) : 

        """"
        In order to fix the architecture of the estimator .. num_layers , cell_type etc ...

        """

        self.estimator = DeepAREstimator(freq=freq, 
                                context_length=context_length,  
                                prediction_length=prediction_length,
                                num_layers=num_layers,    
                                num_cells=num_cells,
                                cell_type=cell_type,
                                #trainer=Trainer(epochs=epochs,ctx=gpu())) 
                                trainer=Trainer(epochs=epochs),
                                use_feat_dynamic_real=True*use_external,
                                use_feat_static_cat=True*use_external,
                                use_feat_static_real=True*use_external,
                                distr_output = NegativeBinomialOutput(),
                                cardinality = cardinality) 

        self.prediction_length = prediction_length

    def train(self) : 

        """
        to do the fit

        """
        self.predictor = self.estimator.train(training_data=self.training_data)

    def evaluate (self,num_samples=100) : 

        """ 
        to do evaluation task

        """

        forecast_it, ts_it = make_evaluation_predictions(
        dataset=self.test_data,  
        predictor=self.predictor,   
        num_samples=num_samples, 
        )
        self.forecasts = list(forecast_it)
        self.tss =  list(ts_it)


    def plot_prob_forecasts(self, index):

        """
        plot the forecasts

        """
        print('hello')
        prediction_intervals = (80.0, 95.0)
        legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]
        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        self.tss[index].plot(ax=ax) 
        self.forecasts[index].plot(color='g')
        #self.forecasts[index].plot(prediction_intervals=prediction_intervals, color='g')
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.show()
         

In [32]:
deepar = DeepAR(dd)
deepar.prepare_train(date['date_start_train'],date['date_end_train_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
cardinality = np.array(dd[cols_stat_cat].max()+1)

deepar.estimator(freq = 'd',context_length = 45, prediction_length = 14, num_layers = 2, num_cells = 40,cell_type = "gru",epochs = 50,cardinality = cardinality.tolist())
deepar.train()

date['date_start_test_mesos'] = '2022-12-01'

deepar.prepare_test(date['date_start_test_mesos'],date['date_end_test_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
deepar.evaluate(num_samples=100)

100%|████████████████████████████████████████████████| 50/50 [00:32<00:00,  1.53it/s, epoch=50/50, avg_epoch_loss=2.38]


In [24]:
def rmsle(y_pred,y):
    return np.sqrt(np.mean((np.log(y+1)-np.log(y_pred+1))**2))

In [33]:
y_pred = deepar.forecasts[0].samples.mean(axis = 0)
y_real = np.array(deepar.tss[0]).reshape(-1)[1:]

In [23]:
warnings.filterwarnings("ignore")

In [34]:
test = pd.DataFrame()
f =df_prova.groupby(df_prova.Dates).size()
test['Agressions'] = f[-14:].to_frame()

In [35]:
test['Pred'] = y_pred

In [30]:
def sqr_err(y_true, y_pred):
    """

    :param y_true: true values of y
    :param y_pred: predicted values of y
    :return: array of lenght original data containing mean squared error for each predictions
    """
    if len(y_true) != len(y_pred):
        raise IndexError("Mismathced array sizes, you inputted arrays with sizes {} and {}".format(len(y_true),
                                                                                                  len(y_pred)))
    else:
        length = len(y_true)

    sqrerror_out = [(y_pred[i]-y_true[i])**2 for i in range(length)]

    return np.array(sqrerror_out)

In [31]:
test['Mean model'] = test['Agressions'].mean()
test['MAPE'] = np.mean(np.abs((test['Agressions'] - test['Pred'])/test['Agressions']))*100
test['MAPE mean'] = np.mean(np.abs((test['Agressions'] - test['Mean model'])/test['Agressions']))*100
test['SMAPE'] = np.mean(np.abs((test['Pred']-test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['SMAPE mean'] = np.mean(np.abs((test['Mean model'] - test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['MAE'] = np.mean(np.abs(test['Agressions'] - test['Pred']))
test['MAE mean'] = np.mean(np.abs(test['Agressions'] - test['Mean model']))
test['Metrica nomb'] = (test['Agressions'] < test['Pred']).sum()/len(test['Pred'])*100
test['Metrica nomb mean'] = (test['Agressions'] < test['Mean model']).sum()/len(test['Pred'])*100
test['RMSE'] = np.sqrt(mean_squared_error(test['Agressions'], test['Pred']))
test['RMSE mean'] = np.sqrt(mean_squared_error(test['Agressions'], test['Mean model']))
test['MSE'] = sqr_err(test['Agressions'], test['Pred'])
test

,Agressions,Pred,Mean model,MAPE,MAPE mean,SMAPE,SMAPE mean,MAE,MAE mean,Metrica nomb,Metrica nomb mean,RMSE,RMSE mean,MSE
Dates,,,,,,,,,,,,,,
2023-01-24,9,4.08,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,24.206401
2023-01-25,4,4.39,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,0.152100
2023-01-26,4,4.74,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,0.547600
2023-01-27,12,4.74,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,52.707603
2023-01-28,5,5.34,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,0.115600
2023-01-29,4,5.48,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,2.190400
2023-01-30,4,4.50,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,0.250000
2023-01-31,5,4.30,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,0.490000
2023-02-01,8,12.08,5.857143,29.592857,33.718821,31.075912,33.928647,2.016429,1.959184,64.285714,64.285714,2.837357,2.386185,16.646399


In [56]:
test['Mean model'] = test['Agressions'].mean()
test['MAPE'] = np.mean(np.abs((test['Agressions'] - test['Pred'])/test['Agressions']))*100
test['MAPE mean'] = np.mean(np.abs((test['Agressions'] - test['Mean model'])/test['Agressions']))*100
test['SMAPE'] = np.mean(np.abs((test['Pred']-test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['SMAPE mean'] = np.mean(np.abs((test['Mean model'] - test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['MAE'] = np.mean(np.abs(test['Agressions'] - test['Pred']))
test['MAE mean'] = np.mean(np.abs(test['Agressions'] - test['Mean model']))
test['Metrica nomb'] = (test['Agressions'] < test['Pred']).sum()/len(test['Pred'])*100
test['Metrica nomb mean'] = (test['Agressions'] < test['Mean model']).sum()/len(test['Pred'])*100
test['RMSE'] = np.sqrt(mean_squared_error(test['Agressions'], test['Pred']))
test['RMSE mean'] = np.sqrt(mean_squared_error(test['Agressions'], test['Mean model']))
test['MSE'] = sqr_err(test['Agressions'], test['Pred'])
test

,Agressions,Pred,Mean model,MAPE,MAPE mean,SMAPE,SMAPE mean,MAE,MAE mean,Metrica nomb,Metrica nomb mean,RMSE,RMSE mean,MSE
Dates,,,,,,,,,,,,,,
2023-01-24,9,4.39,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,21.252101
2023-01-25,4,4.07,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.004900
2023-01-26,4,4.33,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.108900
2023-01-27,12,4.33,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,58.828901
2023-01-28,5,4.05,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.902500
2023-01-29,4,4.39,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.152100
2023-01-30,4,4.09,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.008100
2023-01-31,5,4.34,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,0.435600
2023-02-01,8,12.69,5.857143,23.578968,33.718821,27.809583,36.370431,1.877143,1.959184,50.0,64.285714,2.971077,2.386185,21.996096


In [88]:
prova_bona = test

In [89]:
deepar_bona = prova_bona.to_csv('deepar_bona.csv')

In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(deepar.tss, deepar.forecasts)
agg_metrics